In [ ]:
import random
from functools import reduce  
from operator import add
import numpy as np
from scipy import stats
import matplotlib.pylab as plt 
%matplotlib inline
import matplotlib as mpl
import tqdm
plt.style.use('seaborn-white')
mpl.pyplot.set_cmap('hot')

This document contains a small tutorial on entropy and testing for randomness. 

When I go on holiday I always bring two card games that I can play with my girlfriend: [love letters]() and [sushi go](). The love letters game only consists of 16 cards and my girlfriend noticed that it was rather common that the same sets of cards occured even after shuffling. She though it was because of the way that I was shuffling cards. We got into a bit of a discussion on how one should shuffle cards, which led me to investigate it in a notebook. 

## Methods of Shuffling

I will be considering:

- the one way shuffle: the deck is kept in one hand and blocks of cards are slid unto the other hand making sure the top block of the main hand is put on top of the other hand. 
- the two way shuffle: the deck is kept in one hand and blocks of cards are slid unto the other hand alternating between stacking above and below 
- the noisy split shuffle: split the deck and zip the two stacks, we'll assume some noise over an otherwise perfect split. 

## Measurements of randomness 

#### Entropy 

#### Binomials 

## Results 

- depending on deck size 
- depending on types of shuffle 
- over number if shuffles 
- discussion 

## Insights 

- try out different orders of shuffling 
- is there a best way to shuffle?

## Conclusion 

In [ ]:
class Deck:
    def __init__(self, n_cards=52):
        self.n_cards = n_cards
        self.cards = list(range(n_cards))
    
    def one_way_shuffle(self, n_times=1, at_a_time=[2,3,4,5,6,7]):
        cards = self.cards
        for i in range(n_times):
            new_cards = [] 
            num_cards = len(cards)
            i = 0
            while len(new_cards) < num_cards: 
                tick_size = random.choice(at_a_time)
                new_batch, cards = cards[:tick_size], cards[tick_size:]
                new_cards = new_batch + new_cards
                i += 1
            cards = new_cards
        self.cards = new_cards
        return self 
    
    def two_way_shuffle(self, n_times=1, at_a_time=[2,3,4,5,6,7]):
        cards = self.cards
        for i in range(n_times):
            new_cards = [] 
            num_cards = len(cards)
            i = 0
            while len(new_cards) < num_cards: 
                tick_size = random.choice(at_a_time)
                new_batch, cards = cards[:tick_size], cards[tick_size:]
                new_cards = new_cards + new_batch if i % 2 == 0 else new_batch + new_cards
                i += 1
            cards = new_cards
        self.cards = new_cards
        return self 
    
    def split_shuffle(self, n_times=1, p_flip=0):
        split_idx = round(self.n_cards/2)
        for i in range(n_times):
            top = self.cards[split_idx:]
            bottom = self.cards[:split_idx]
            self.cards = reduce(add, [[a] + [b] for a,b in zip(top, bottom)])
            for i in range(round(self.n_cards*p_flip)):
                idx = random.choice(range(self.n_cards))
                old_cards = self.cards[:]
                old_cards[idx], old_cards[idx-1] = old_cards[idx-1], old_cards[idx]
                self.cards = old_cards
        return self 
    
    def shuffle(self, typename, *args, **kwargs):
        if typename == "one_way":
            return self.one_way_shuffle(*args, **kwargs)
        if typename == "two_way":
            return self.two_way_shuffle(*args, **kwargs)
        if typename == "split":
            return self.split_shuffle(*args,**kwargs, p_flip=0)
        if typename == "noisy_split":
            return self.split_shuffle(*args,**kwargs, p_flip=0.1)
        assert False

In [ ]:
assert len(Deck(n_cards=10).split_shuffle().cards) == 10 
assert len(Deck(n_cards=20).split_shuffle(p_flip=0.1).cards) == 20 
assert len(Deck(n_cards=12).one_way_shuffle(n_times=1).cards) == 12
assert len(Deck(n_cards=12).two_way_shuffle(n_times=1).cards) == 12

In [ ]:
def calc_qx(shuffles):
    qx = np.zeros(shape=(len(shuffles[0]), len(shuffles[0])))
    for simulation in shuffles:
        for j, placement in enumerate(simulation):
            qx[j, placement] += 1 
    return qx


def calc_entropy(shuffelds):
    entropies = []
    for i in calc_qx(shuffelds):
        p = i/i.sum()
        logp = np.log2(p)
        logp[p == 0] = 0
        entropies.append(np.sum(p*logp))
    return -np.mean(entropies)


def calc_kl(shuffelds):
    entropies = []
    for i in calc_qx(shuffelds):
        q = i/i.sum() + 0.0000001
        p = 1/len(shuffelds[0])
        logpq = np.log2(p/q)
        entropies.append(np.sum(p*logpq))
    return np.mean(entropies)

In [ ]:
def varprint(*args, **kwargs):
    output = ""
    for arg in args: 
        output += f"{args}"
    for key, val in kwargs.items():
        output += f" {key}:{val}"
    print(output)
    
def run_simulation(shuffle_type, n_cards, n_times, n_iter=10000):
    sim_shuffled_indices = []
    for i in range(n_iter):
        res = (Deck(n_cards=n_cards)
                     .shuffle(shuffle_type, n_times=n_times)
                     .cards)
        sim_shuffled_indices.append(res)
    return sim_shuffled_indices

In [ ]:
n_iter = 10000
for n_cards in [16, 52, 104]:
    for shuffle_type in ['one_way', 'two_way', 'noisy_split',]:
        for n_times in range(1, 25+1):
            sim_shuffled_indices = run_simulation(shuffle_type, n_cards, n_times, n_iter=10000)

            distrib = stats.binom(n=n_iter, p=1/n_cards)
            qx = calc_qx(sim_shuffled_indices)
            n_failed_tests = np.sum(distrib.cdf(qx) < 0.01) + np.sum(distrib.cdf(qx) > 0.99)
            entropy = round(calc_entropy(sim_shuffled_indices),3)
            relative_entropy = round(np.log2(n_cards),3)

            plt.figure(figsize=(6,6))
            plt.imshow(np.flip(np.array([i/i.sum() for i in calc_qx(sim_shuffled_indices)]), axis=0),
                                 interpolation='none', extent=[0, n_cards, 0, n_cards],
                                 cmap=plt.cm.coolwarm, vmin=0, vmax=3/n_cards)
            plt.colorbar()
            plt.title(f"type: {shuffle_type}-{n_times} entr: {entropy}/{relative_entropy} fails: {n_failed_tests}/{n_cards*n_cards}");
            plt.savefig(f"shuffles/{shuffle_type}-{n_cards}-{str(n_times).zfill(2)}-{n_iter}.png", bbox_inches='tight')
            varprint(type=shuffle_type, n_cards=n_cards, n_times=n_times, n_iter=n_iter, entropy=entropy, status="saved.")

At least from eyeballing: it seems that after 3 shuffles, it isn't that bad. Can we quantify how valid it is though? It would be nice if we have some sort of result depending on the number of shuffles and the number of cards. 
We could calculate the entropy of the estimated probabilities. The higher the entropy, the better. 

Interestingly enough, we have a maximum possible entropy given $N$ cards. 

$$ H[p(x)] = - \sum_i p_i \log p_i = - \sum_i \frac{1}{N} \log \frac{1}{N} = \sum_{i=1}^N \frac{1}{N} \log N = \log N$$ 

This means that we can check; what order of practical shuffling actions maximises the entropy?

Entropy is a measure of randomness but it doesn't test for randomness. As well as I expected.


```
convert -loop 0 -delay 50 noisy_split-52-*-10000.png noisy.gif
convert -loop 0 -delay 50 one_way-52-*-10000.png oneway.gif
convert -loop 0 -delay 50 two_way-52-*-10000.png twoway.gif
```

In [ ]:
n_iter = 5000
data = [] 
for n_cards in [16, 32, 52, 128]:
    for shuffle_type in ['noisy_split', 'one_way', 'two_way', ]:
        for n_times in range(1, 25 + 1):
            sim_shuffled_indices = run_simulation(shuffle_type, n_cards, n_times, n_iter=n_iter)
            distrib = stats.binom(n=n_iter, p=1/n_cards)
            qx = calc_qx(sim_shuffled_indices)
            n_failed_tests = np.sum(distrib.cdf(qx) < 0.01) + np.sum(distrib.cdf(qx) > 0.99)

            distrib = stats.binom(n=n_iter, p=1/n_cards)
            qx = calc_qx(sim_shuffled_indices)
            n_failed_tests = np.sum(distrib.cdf(qx) < 0.01) + np.sum(distrib.cdf(qx) > 0.99)
            entropy = round(calc_entropy(sim_shuffled_indices), 3)
            relative_entropy = round(np.log2(n_cards), 3)
            kl = calc_kl(sim_shuffled_indices)
            data.append(dict(type=shuffle_type, 
                     n_cards=n_cards, 
                     n_times=n_times, 
                     n_iter=n_iter, 
                     entropy=entropy, 
                     failed_tests=n_failed_tests, 
                     kl=kl))
            varprint(type=shuffle_type, 
                     n_cards=n_cards, 
                     n_times=n_times, 
                     n_iter=n_iter, 
                     entropy=entropy, 
                     failed_tests=n_failed_tests, 
                     kl=kl)

In [ ]:
import pandas as pd 
from plotnine import ggplot, aes, geom_line, facet_grid, ggtitle

df = (pd.DataFrame(data)
     .assign(p_fail = lambda d: d.failed_tests/d.n_cards/d.n_cards)
     .assign(n_cards = lambda d: d.n_cards.apply(str)))
df.head()

In [ ]:
# df

In [ ]:
(ggplot() + 
  geom_line(data=df, mapping=aes('n_times', 'p_fail')) + 
  facet_grid('n_cards ~ type'))